In [1]:
import pandas as pd
import shutil
from tqdm import tqdm
import os
from torchvision import transforms
import glob
from PIL import Image
import numpy as np

In [2]:
csv = pd.read_csv('data/train.csv')

In [3]:
for i in tqdm(range(len(csv))):
    directory = 'data/train/'+str(csv.iloc[i][1])
    if not os.path.exists(directory):
        os.mkdir(directory)
    shutil.move('data/train/'+csv.iloc[i][0],directory)

100%|██████████| 22912/22912 [00:03<00:00, 5959.09it/s]


In [4]:
!cp -r data data-nong

## 增广

In [5]:
def normal(src):
    # 定义增广方法,需要根据任务以及图像适当做调整
    trains_list = [
        transforms.Compose([
            transforms.RandomHorizontalFlip(p=1),
            transforms.ToTensor(),
    ]),
        transforms.Compose([
            transforms.RandomVerticalFlip(p=1),
                transforms.ToTensor(),
    ]),
        transforms.Compose([
            transforms.RandomRotation(20),
            transforms.ToTensor(),
    ]),
        transforms.Compose([
            transforms.ColorJitter(brightness=0.4, contrast=0.3, saturation=0.2, hue=0.3),  # 随机调整亮度，对比度，饱和度和色相
            transforms.ToTensor(),
    ]),
    #     transforms.Compose([
    #         transforms.ToTensor(),
    #         transforms.RandomErasing(p=1, scale=(0.02, 0.1), ratio=(0.3, 0.3), value=(0, 0, 0)),
    # ]),
    #     transforms.Compose([
    #         transforms.Resize((1024, 1024)),
    #         transforms.RandomCrop(size=512, padding=0, pad_if_needed=False, fill=0, padding_mode='constant'),
    #         transforms.ToTensor(),
    # ])
    ]
    all_items = os.listdir(src) #data/train/

    for h in tqdm(range(len(all_items))):
        data_dir = src + all_items[h]
        imgs_names = glob.glob(data_dir + '/*.jpg')
        count = 0
        for i in range(len(imgs_names)):
            image = Image.open(imgs_names[i]).convert('RGB')
            for trans in trains_list:
                aug_image = trans(image)
                count += 1
                filename = f'{all_items[h]}_image_{count}.jpg'
                save_path = os.path.join("data/train/"+all_items[h], filename)
                aug_image_pil = transforms.ToPILImage()(aug_image)
                aug_image_pil.save(save_path)

In [6]:
normal("data-nong/train/")

100%|██████████| 25/25 [01:09<00:00,  2.79s/it]


In [7]:
# 随机擦除
def random_erase(img,n_holes,length,rate): #输入img为PIL图片格式的图片
    if np.random.rand(1)[0]<rate:
        img = np.array(img)
        h = img.shape[0] #图片的高
        w = img.shape[1] #图片的宽
        
        n_holes = np.random.randint(n_holes)
        mask = np.ones((h, w), np.float32) #32*32w*h的全1矩阵

        for n in range(n_holes): #n_holes=2,length=4 选择2个区域；每个区域的边长为4
            y = np.random.randint(h) #0~31随机选择一个数 y=4
            x = np.random.randint(w) #0~31随机选择一个数 x=24

            y1 = np.clip(y - length // 2, 0, h) #2,0,32 ->2
            y2 = np.clip(y + length // 2, 0, h) #6,0,32 ->6
            x1 = np.clip(x - length // 2, 0, w) #24-2,0,32 ->22
            x2 = np.clip(x + length // 2, 0, w) #24+2,0,32 ->26

            mask[y1: y2, x1: x2] = 0. #将这一小块区域去除
        img[:,:,0] = img[:,:,0] * mask
        img[:,:,1] = img[:,:,1] * mask
        img[:,:,2] = img[:,:,2] * mask
        return Image.fromarray(img)
    else:
        return img

In [8]:
all_items = os.listdir('data-nong/train')
for i in tqdm(range(len(all_items))):
        data_dir = 'data-nong/train/' + all_items[i]
        imgs_names = glob.glob(data_dir + '/*.jpg')
        count = 0
        for j in range(len(imgs_names)):
            img = Image.open(imgs_names[j]).convert('RGB')
            img2 = random_erase(img,150,3,1)
            count += 1
            img2.save("data/train/"+all_items[i]+"/erase"+str(count)+".jpg")

100%|██████████| 25/25 [01:24<00:00,  3.36s/it]


## 为标签

In [13]:
# csv1 = pd.read_csv('1.csv')
# for i in tqdm(range(len(csv1))):
#     shutil.copy('data/test/'+csv1.iloc[i][0],'data/train/'+str(csv1.iloc[i][1]))

100%|██████████| 6165/6165 [00:02<00:00, 2343.97it/s]


In [11]:
import torch.utils.data.distributed
import torchvision.transforms as transforms

from torch.autograd import Variable
from PIL import Image
import pandas as pd
import glob
import torch.nn as nn
from torchvision import models
import timm
from tqdm import tqdm

path_pre_img = pd.Series(glob.glob('data/test/*'))
path_pre_img = path_pre_img.apply(lambda x:x.split('/')[2]).tolist()

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.4, contrast=0.3, saturation=0.2, hue=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test4 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
list_trans = [transform_test,transform_test1,transform_test2,transform_test3,transform_test4]
# model = SVHN_Model2().cuda()
import timm
model = timm.create_model('convnext_small_in22k', pretrained=True,num_classes=25).cuda()
model.load_state_dict(torch.load("best_model-3.pth"))

classes=['0','1','10','11','12','13','14','15','16','17','18','19','2','20','21','22','23','24','3','4','5','6','7','8','9']
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(DEVICE)
path='data/test/'
for i in tqdm(range(len(path_pre_img))):
    for j in range(len(list_trans)):
        img=Image.open(path+path_pre_img[i])
        img=img.convert('RGB')
        img=list_trans[j](img)
        img.unsqueeze_(0)
        img = Variable(img).to(DEVICE)
        if j==0:
            out=model(img)
        else:
            out+=model(img)
    img=Image.open(path+path_pre_img[i])
    img=img.convert('RGB')
    img=random_erase(img,150,3,1)
    img=transform_test(img)
    img.unsqueeze_(0)
    img = Variable(img).to(DEVICE)
    out+=model(img)
    probs = torch.nn.functional.softmax(out, dim=-1)
    confidence, predicted_class = torch.max(probs, 1)
    confidence = confidence.item()
    _, pred = torch.max(out.data, 1)
    if confidence > 0.9:
        shutil.copy('data/test/'+path_pre_img[i],'data/train/'+classes[pred.data.item()])

/mistgpu/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(
100%|██████████| 6165/6165 [06:58<00:00, 14.74it/s]


## 预测

In [4]:
import torch.utils.data.distributed
import torchvision.transforms as transforms

from torch.autograd import Variable
from PIL import Image
import pandas as pd
import glob
import torch.nn as nn
from torchvision import models
import timm
import numpy as np
from tqdm import tqdm

path_pre_img = pd.Series(glob.glob('data/test/*'))
path_pre_img = path_pre_img.apply(lambda x:x.split('/')[2]).tolist()

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.4, contrast=0.3, saturation=0.2, hue=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
list_trans = [transform_test,transform_test1,transform_test2,transform_test3]

In [5]:
# 随机擦除
def random_erase(img,n_holes,length,rate): #输入img为PIL图片格式的图片
    if np.random.rand(1)[0]<rate:
        img = np.array(img)
        h = img.shape[0] #图片的高
        w = img.shape[1] #图片的宽
        
        n_holes = np.random.randint(n_holes)
        mask = np.ones((h, w), np.float32) #32*32w*h的全1矩阵

        for n in range(n_holes): #n_holes=2,length=4 选择2个区域；每个区域的边长为4
            y = np.random.randint(h) #0~31随机选择一个数 y=4
            x = np.random.randint(w) #0~31随机选择一个数 x=24

            y1 = np.clip(y - length // 2, 0, h) #2,0,32 ->2
            y2 = np.clip(y + length // 2, 0, h) #6,0,32 ->6
            x1 = np.clip(x - length // 2, 0, w) #24-2,0,32 ->22
            x2 = np.clip(x + length // 2, 0, w) #24+2,0,32 ->26

            mask[y1: y2, x1: x2] = 0. #将这一小块区域去除
        img[:,:,0] = img[:,:,0] * mask
        img[:,:,1] = img[:,:,1] * mask
        img[:,:,2] = img[:,:,2] * mask
        return Image.fromarray(img)
    else:
        return img

In [6]:
# model = SVHN_Model2().cuda()
import timm
model = timm.create_model('convnext_small_in22k', pretrained=True,num_classes=25).cuda()
model.load_state_dict(torch.load("weights/best_model-3.pth"))

classes=['0','1','10','11','12','13','14','15','16','17','18','19','2','20','21','22','23','24','3','4','5','6','7','8','9']
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(DEVICE)
path='data/test/'
pre_data =[]
pre_label=[]
for i in tqdm(range(len(path_pre_img))):
    for j in range(len(list_trans)):
        img=Image.open(path+path_pre_img[i])
        img=img.convert('RGB')
        img=list_trans[j](img)
        img.unsqueeze_(0)
        img = Variable(img).to(DEVICE)
        if j==0:
            out=model(img)
        else:
            out+=model(img)
    # Predict
    img=Image.open(path+path_pre_img[i])
    img=img.convert('RGB')
    img=random_erase(img,150,3,1)
    img=transform_test(img)
    img.unsqueeze_(0)
    img = Variable(img).to(DEVICE)
    out+=model(img)
    _, pred = torch.max(out.data, 1)
    pre_data.append(path_pre_img[i])
    pre_label.append(classes[pred.data.item()])

sub = pd.DataFrame()
sub['name'] = pre_data
sub['label'] = pre_label
sub.to_csv('./pre_submit.csv',index=False)

100%|██████████| 6165/6165 [05:35<00:00, 18.39it/s]
